В этом задании начнем с простого изучения данных.

Необходимо посмотреть на данные и разобраться с ними: 
1. Выявить порядковые категориальные данные.
2. Посмотреть на сатистики по данным.
3. Удостовериться, что данные не противоречат логике.

 Как делать первый пункт:
     * Описать, как ты понимаешь каждую колонку. Колонки с текстами, картинками и id сейчас не рассматриваем. 
     * Доказать, что колонка является просто категориальной или порядковой категориальной.
     * Просто категориальные оставляем, а порядковые проверяем на логичность нумерации. В случае нелогичности перенумеровываем.
     * Как проверяем порядковые. Смотрим по логике, напрмер, рассматриваем, как соотносятся размеры пород с с размерами в колонке размеров.
     
Как делать второй пункт:
    * Посмотреть и визуализировать, например, сколько каких животных по возрасту.
    * Или сколько каким типам животных нужно для того, чтобы из забрали.
    
Третий пункт вытекает из первых двух, обобщение собранной информации. 


### Примеры кода, который может быть полезен

In [ ]:
import pandas as pd

In [ ]:
breeds = pd.read_csv('../input/breed_labels.csv')
colors = pd.read_csv('../input/color_labels.csv')
states = pd.read_csv('../input/state_labels.csv')
train = pd.read_csv('../input/train/train.csv')
test = pd.read_csv('../input/test/test.csv')

train.head()

In [ ]:
breed_dict = dict(zip(breeds['BreedID'], breeds['BreedName']))
# тут пишут, что есть более быстрый способ, но интуитивно менее понятный:
# https://stackoverflow.com/questions/17426292/what-is-the-most-efficient-way-to-create-a-dictionary-of-two-pandas-dataframe-co

# pd.Series(breeds['BreedName'].values, index=breeds['BreedID']).to_dict()

In [ ]:
animal_dict = {1: 'Dog', 2: 'Cat'}

In [ ]:
train['Breed1_new'] = train['Breed1'].map(breed_dict)
train['Type'] = train['Type'].map(animal_dict)

In [ ]:
train.groupby(['Type', 'Breed1_new', 'FurLength'])['PetID'].count()

Забавно, видно, что есть одни и те же породы животных с разной длиной шерсти.  На самом деле, возможно, стоит отсмотреть фотографии животных, где такое есть и поставить ручные метки, потому что реальные могут оказаться не совсем корректными. Вот и первый инсайд.

Еще пока крутила данные, пришла к выводу, что хорошим признаком может быть среднерыночная стоимость животных. Так как тех же акит не так уж и много. Так что, возможно, стоит попарсить сайты с ценами на животных. Вот и второй инсайд.

### Еще немного важного, что обсуждали на занятии:
    * Метрики: немного про них тут https://github.com/esokolov/ml-course-hse/blob/master/2017-fall/lecture-notes/lecture02-linregr.pdf
    Можно использовать, как источник ключевых слов.
    * Градиентный бустинг: https://habr.com/en/company/ods/blog/327250/

**Решение**

**Фичи**

Категориальные фичи:
1. Vaccinated - статус привитости (1 = Да, 2 = Нет, 3 = Неизвестно)
2. Dewormed - статус применения противоглистных препаратов (1 = Да, 2 = Нет, 3 = Неизвестно)
3. Sterilized - статус стерилизации (1 = Да, 2 = Нет, 3 = Неизвестно)
4. Health - здоровье питомца (1 = Здоровый, 2 = Незначительная травма, 3 = Серьезная травма, 0 = Неизвестно)

Порядковые категориальные фичи:
1. MaturitySize - размер питомца (1 = Маленький, 2 = Средний, 3 = Большой, 4 = Очень большой, 0 = Неизвестно). Видно, если посмотреть фотографии пород в сети.
2. FurLength - длина шерсти питомца (1 = Короткая, 2 = Средняя, 3 = Длинная, 0 = Неизвестно). Видно, если посмотреть фотографии пород в сети.

In [ ]:
#Разделим датафрейм на кошек и собак и рассмотрим их раздельно:
train_cats = train[train.Type != 'Dog']
train_dogs = train[train.Type != 'Cat']

#Для каждого распределения попробуем построить визуализацию
import seaborn as sns

***Кошки***

In [ ]:
#Создаем для кошек датафреймы для описания распределений скорости adoption в разрезах различных категориальных фич
ms_train_cats = pd.DataFrame(train_cats.groupby([ 'AdoptionSpeed', 'MaturitySize'])['PetID'].count()).reset_index()
fr_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'FurLength'])['PetID'].count()).reset_index()
vc_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Vaccinated'])['PetID'].count()).reset_index()
dw_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Dewormed'])['PetID'].count()).reset_index()
st_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Sterilized'])['PetID'].count()).reset_index()
h_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Health'])['PetID'].count()).reset_index()
age_train_cats = pd.DataFrame(train_cats.groupby(['AdoptionSpeed', 'Age'])['PetID'].count()).reset_index()

In [ ]:
#Добавляем колонку в датафреймы с разрезами с долей питомцев от общего числа для каждого значения категориальной фичи
ms_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
fr_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
vc_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
dw_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
st_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
h_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993
age_train_cats['PetID_Share'] = ms_train_cats['PetID']/14993

**Размер**

In [ ]:
#распределение размеров по количеству питомцев
train_cats.groupby(['MaturitySize'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['MaturitySize'])

In [ ]:
#распределение размеров питомцев в разрезе adoption скорости
train_cats.groupby(['MaturitySize', 'AdoptionSpeed'])['PetID'].count()

In [ ]:
# Хитмап по размеру питомца
pv_ms_train_cats = ms_train_cats.pivot_table(values='PetID_Share', index='MaturitySize',   columns='AdoptionSpeed')
sns.heatmap(pv_ms_train_cats, cmap='inferno_r')

Выводы:
1.     Больше всего кошек среднего размера (примерно в два арза больше, чем кошек маленького размера).
2.     На хитман диаграмме видно, что кошек маленького и большого размера забирают примерно с одной скоростью (учитывая разницу в количестве питомцев этих размеров).
3.     Больше всего забирают кошек средних размеров. При этом скорость их забора менее равномерна, чем у кошек маленького и большого размера (см. хитман диаграмму).
4.     При этом с примерно равной вероятностью кошку среднего размера забирают в течение первого месяца, либо не забирают вообще.

**Длина шерсти**

In [ ]:
#распределение длины шерсти по количеству питомцев
train_cats.groupby(['FurLength'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['FurLength'])

In [ ]:
#распределение длины шерсти питомцев в разрезе adoption скорости
train_cats.groupby(['AdoptionSpeed', 'FurLength'])['PetID'].count()

In [ ]:
#Хитмап по длине шерсти питомца
pv_fr_train_cats = fr_train_cats.pivot_table(values='PetID_Share', index='FurLength',   columns='AdoptionSpeed')
sns.heatmap(pv_fr_train_cats, cmap='inferno_r')

Выводы:
    

**Статус вакцинации**

In [ ]:
#распределение статуса вакцинации по количеству питомцев
train_cats.groupby(['Vaccinated'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['Vaccinated'])

In [ ]:
#распределение статуса вакцинации в разрезе adoption скорости
train_cats.groupby(['AdoptionSpeed', 'Vaccinated'])['PetID'].count()

In [ ]:
pv_vc_train_cats = vc_train_cats.pivot_table(values='PetID_Share', index='Vaccinated',   columns='AdoptionSpeed')
sns.heatmap(pv_vc_train_cats, cmap='inferno_r')

Выводы:


**Cтатус применения противоглистных препаратов**

In [ ]:
#распределение статусов применения противоглистных препаратов по количеству питомцев
train_cats.groupby(['Dewormed'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['Dewormed'])

In [ ]:
#распределение статуса применения противоглистных препаратов в разрезе adoption скорости
train_cats.groupby(['AdoptionSpeed', 'Dewormed'])['PetID'].count()

In [ ]:
pv_dw_train_cats = dw_train_cats.pivot_table(values='PetID_Share', index='Dewormed',   columns='AdoptionSpeed')
sns.heatmap(pv_dw_train_cats, cmap='inferno_r')

Выводы:


**Статус стерилизации**

In [ ]:
#распределение статусов стерилизации по количеству питомцев
train_cats.groupby(['Sterilized'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['Sterilized'])

In [ ]:
#распределение статуса стерилизации в разрезе adoption скорости
train_cats.groupby(['AdoptionSpeed', 'Sterilized'])['PetID'].count()

In [ ]:
pv_st_train_cats = st_train_cats.pivot_table(values='PetID_Share', index='Sterilized',   columns='AdoptionSpeed')
sns.heatmap(pv_st_train_cats, cmap='inferno_r')

Выводы:
    

**Здоровье**

In [ ]:
#распределение статусов здоровья по количеству питомцев
train_cats.groupby(['Health'])['PetID'].count()

In [ ]:
sns.distplot(train_cats['Health'])

In [ ]:
#распределение статуса здоровья в разрезе adoption скорости
train_cats.groupby(['AdoptionSpeed', 'Health'])['PetID'].count()

In [ ]:
pv_h_train_cats = h_train_cats.pivot_table(values='PetID_Share', index='Health',   columns='AdoptionSpeed')
sns.heatmap(pv_h_train_cats, cmap='inferno_r')

Выводы:


**Возраст**

In [ ]:
sns.distplot(train_cats['Age'])

In [ ]:
pv_age_train_cats = age_train_cats.pivot_table(values='PetID_Share', index='Age',   columns='AdoptionSpeed')
sns.heatmap(pv_age_train_cats, cmap='inferno_r')

Выводы:
    

**Собаки****

In [ ]:
#Создаем для собак датафреймы для описания распределений скорости adoption в разрезах различных категориальных фич
ms_train_dogs = pd.DataFrame(train_dogs.groupby([ 'AdoptionSpeed', 'MaturitySize'])['PetID'].count()).reset_index()
fr_train_dogs = pd.DataFrame(train_dogs.groupby(['AdoptionSpeed', 'FurLength'])['PetID'].count()).reset_index()
vc_train_dogs = pd.DataFrame(train_dogs.groupby(['AdoptionSpeed', 'Vaccinated'])['PetID'].count()).reset_index()
dw_train_dogs = pd.DataFrame(train_dogs.groupby(['AdoptionSpeed', 'Dewormed'])['PetID'].count()).reset_index()
st_train_dogs = pd.DataFrame(train_dogs.groupby(['AdoptionSpeed', 'Sterilized'])['PetID'].count()).reset_index()
h_train_dogs = pd.DataFrame(train_dogs.groupby(['AdoptionSpeed', 'Health'])['PetID'].count()).reset_index()
age_train_dogs = pd.DataFrame(train_dogs.groupby(['AdoptionSpeed', 'Age'])['PetID'].count()).reset_index()

In [ ]:
#Добавляем колонку в датафреймы с разрезами с долей питомцев от общего числа для каждого значения категориальной фичи
ms_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993
fr_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993
vc_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993
dw_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993
st_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993
h_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993
age_train_dogs['PetID_Share'] = ms_train_dogs['PetID']/14993

**Размер**

In [ ]:
#распределение размеров по количеству питомцев
train_dogs.groupby(['MaturitySize'])['PetID'].count()

In [ ]:
sns.distplot(train_dogs['MaturitySize'])

In [ ]:
#распределение размеров питомцев в разрезе adoption скорости
train_dogs.groupby(['MaturitySize', 'AdoptionSpeed'])['PetID'].count()

In [ ]:
# Хитмап по размеру питомца
pv_ms_train_dogs = ms_train_dogs.pivot_table(values='PetID_Share', index='MaturitySize',   columns='AdoptionSpeed')
sns.heatmap(pv_ms_train_dogs, cmap='inferno_r')

Выводы:
    

**Длина шерсти**

In [ ]:
#распределение длины шерсти по количеству питомцев
train_dogs.groupby(['FurLength'])['PetID'].count()

In [ ]:
sns.distplot(train_dogs['FurLength'])

In [ ]:
#распределение длины шерсти питомцев в разрезе adoption скорости
train_dogs.groupby(['AdoptionSpeed', 'FurLength'])['PetID'].count()

In [ ]:
#Хитмап по длине шерсти питомца
pv_fr_train_dogs = fr_train_dogs.pivot_table(values='PetID_Share', index='FurLength',   columns='AdoptionSpeed')
sns.heatmap(pv_fr_train_dogs, cmap='inferno_r')

Выводы:


**Статус вакцинации**

In [ ]:
#распределение статуса вакцинации по количеству питомцев
train_dogs.groupby(['Vaccinated'])['PetID'].count()

In [ ]:
sns.distplot(train_dogs['Vaccinated'])

In [ ]:
#распределение статуса вакцинации в разрезе adoption скорости
train_dogs.groupby(['AdoptionSpeed', 'Vaccinated'])['PetID'].count()

In [ ]:
pv_vc_train_dogs = vc_train_dogs.pivot_table(values='PetID_Share', index='Vaccinated', columns='AdoptionSpeed')
sns.heatmap(pv_vc_train_dogs, cmap='inferno_r')

Выводы:


**Cтатус применения противоглистных препаратов**

In [ ]:
#распределение статусов применения противоглистных препаратов по количеству питомцев
train_dogs.groupby(['Dewormed'])['PetID'].count()

In [ ]:
sns.distplot(train_dogs['Dewormed'])

In [ ]:
#распределение статуса применения противоглистных препаратов в разрезе adoption скорости
train_dogs.groupby(['AdoptionSpeed', 'Dewormed'])['PetID'].count()

In [ ]:
pv_dw_train_dogs = dw_train_dogs.pivot_table(values='PetID_Share', index='Dewormed',   columns='AdoptionSpeed')
sns.heatmap(pv_dw_train_dogs, cmap='inferno_r')

Выводы:


**Статус стерилизации**

In [ ]:
#распределение статусов стерилизации по количеству питомцев
train_dogs.groupby(['Sterilized'])['PetID'].count()

In [ ]:
sns.distplot(train_dogs['Sterilized'])

In [ ]:
#распределение статуса стерилизации в разрезе adoption скорости
train_dogs.groupby(['AdoptionSpeed', 'Sterilized'])['PetID'].count()

In [ ]:
pv_st_train_dogs = st_train_dogs.pivot_table(values='PetID_Share', index='Sterilized',   columns='AdoptionSpeed')
sns.heatmap(pv_st_train_dogs, cmap='inferno_r')

Выводы:


**Здоровье**

In [ ]:
#распределение статусов здоровья по количеству питомцев
train_dogs.groupby(['Health'])['PetID'].count()

In [ ]:
sns.distplot(train_dogs['Health'])

In [ ]:
#распределение статуса здоровья в разрезе adoption скорости
train_dogs.groupby(['AdoptionSpeed', 'Health'])['PetID'].count()

In [ ]:
pv_h_train_dogs = h_train_dogs.pivot_table(values='PetID_Share', index='Health',   columns='AdoptionSpeed')
sns.heatmap(pv_h_train_dogs, cmap='inferno_r')

Выводы:

**Возраст**

In [ ]:
sns.distplot(train_dogs['Age'])

In [ ]:
pv_age_train_dogs = age_train_dogs.pivot_table(values='PetID_Share', index='Age',   columns='AdoptionSpeed')
sns.heatmap(pv_age_train_dogs, cmap='inferno_r')

Выводы:
    

**To Do:**
1. Попарсить цены на различные породы с малазийского сайта https://www.petsales.my. Собрать цены в датафрейм вида {"Порода" : Стоимость}. Смерджить со словарем пород и с train. Посмотреть, зависит ли скорость adoption животных
2. Посмотреть, влияет ли размер fee на скорость adoption.
3. Посмотреть, влияет ли штат (малазийский регион) на скорость adoption.
1. Создать новую категориальную фичу Multicolor (аля мультицветность), показывающую количество цветов животного. Возможные значения: 1, 2, 3. Логика заполнения значения value = a + b + c, где
    а)  a\b\c = 0, если Color1\Color2\Color3 = 0 или NaN
    b) a\b\c = 1, если Color1\Color2\Color3 != 0 или NaN.
   Затем посмотреть, зависит ли скорость adoption от количества цветов питомца.